## Load Data

In [3]:
import pandas as pd

df = pd.read_csv("exploit_hacking_set Data Scientist.csv",
                 converters={"type": str, "label": str, "sub_label": str, "text": str, "title": str, "site": str})

In [4]:
df["spont_score"] = df["spont_score"].fillna(value=0.239)
df

,type,label,sub_label,text,title,site,spont_score
0,post,Hacking,None,i haven t seen a post about this rat lately . ...,spynote v android rat,forum_sinisterly,60.869565
1,post,Fraud,None,looking for us partner simple extra money add ...,looking for us partner simple extra money,forum_alphabay,-30.434783
2,post,Fraud,Carding,i have drop address in us eu for who need for ...,for who needs drop address in us eu,forum_alphabay,-108.695652
3,post,None,None,have a closed prefix on the thread names after...,suggestions,forum_sinisterly,-26.086957
4,post,Computer,Coding,hey guys . i was working on an updated mass tw...,php leaks / downloads,forum_leakforums,-4.347826
...,...,...,...,...,...,...,...
10330,,,,,,,0.239000
10331,,,,,,,0.239000
10332,,,,,,,0.239000
10333,,,,,,,0.239000


In [5]:
y_hac = df['label'] == 'Hacking'
y_hac = y_hac.astype(int)
y_exp = df['sub_label'] == 'Exploit'
y_exp = y_exp.astype(int)

y_hac_exp = df['sub_label'] == 'Exploit'
y_hac_exp[y_hac == 1] = 1
y_hac_exp[y_exp == 1] = 2
y_hac_exp = y_hac_exp.astype(int)

In [19]:
df_X = df.drop(columns=['label', 'sub_label'])
if 'zero_shot_tmp' in df_X.columns.values:
    df_X = df_X.drop(columns=['zero_shot_tmp'])

In [11]:
df_X['text_len'] = df_X['text'].apply(len)
df_X['title_len'] = df_X['title'].apply(len)

df_X['text'] = df_X['text'].apply(lambda x: x[:2000])
df_X['title'] = df_X['title'].apply(lambda x: x[:300])


## CatBoost with text features (Original Dataset)

In [22]:
from catboost import CatBoostClassifier
from cross_val import cross_val

### Hacking

In [ ]:
clf_factory = lambda: CatBoostClassifier(iterations=500,
                                         learning_rate=0.2,
                                         loss_function='Logloss',
                                         verbose=False,
                                         text_features=['text', 'title'],
                                         cat_features=['type', 'site'],
                                         feature_calcers=['BoW:top_tokens_count=2000', 'BoW:top_tokens_count=500'],
                                         )
cross_val(clf_factory, df_X, y_hac)
# Cross Validation k_folds=5, n_classes=2:
#         0 pr: 0.8375 rec: 0.7204301075268817 f1: 0.7745664739884393
#         1 pr: 0.8515625 rec: 0.5828877005347594 f1: 0.6920634920634919
#         2 pr: 0.8689655172413793 rec: 0.7078651685393258 f1: 0.7801857585139319
#         3 pr: 0.8306451612903226 rec: 0.5953757225433526 f1: 0.6936026936026936
#         4 pr: 0.8601398601398601 rec: 0.6180904522613065 f1: 0.7192982456140351
# avg: pr=0.8497626077343124 rec=0.6449298302811253 f1=0.7319433327565184


### Exploit

In [ ]:
clf_factory = lambda: CatBoostClassifier(iterations=500,
                                         learning_rate=0.2,
                                         loss_function='Logloss',
                                         verbose=False,
                                         text_features=['text', 'title'],
                                         cat_features=['type', 'site'],
                                         feature_calcers=['BoW:top_tokens_count=2000', 'BoW:top_tokens_count=500'],
                                         )
cross_val(clf_factory, df_X, y_exp)
# Cross Validation k_folds=5, n_classes=2:
#         0 pr: 0.8395061728395061 rec: 0.7906976744186046 f1: 0.8143712574850299
#         1 pr: 0.863013698630137 rec: 0.8289473684210527 f1: 0.8456375838926175
#         2 pr: 0.875 rec: 0.8076923076923077 f1: 0.8400000000000001
#         3 pr: 0.921875 rec: 0.7866666666666666 f1: 0.8489208633093526
#         4 pr: 0.863013698630137 rec: 0.7590361445783133 f1: 0.8076923076923077
# avg: pr=0.872481714019956 rec=0.794608032355389 f1=0.8313244024758616


## CatBoost (Extended with zero-shot features dataset)


### Hacking

In [ ]:
clf_factory = lambda: CatBoostClassifier(iterations=500,
                                         learning_rate=0.2,
                                         loss_function='Logloss',
                                         verbose=False,
                                         text_features=['text', 'title'],
                                         cat_features=['type', 'site', 'zero_shot_1st_label', 'zero_shot_1st_sub_label'],
                                         feature_calcers=['BoW:top_tokens_count=2000', 'BoW:top_tokens_count=500'],
                                         )
cross_val(clf_factory, df_X, y_hac)
# Cross Validation k_folds=5, n_classes=2:
#         0 pr: 0.8571428571428571 rec: 0.7096774193548387 f1: 0.7764705882352942
#         1 pr: 0.8636363636363636 rec: 0.6096256684491979 f1: 0.7147335423197494
#         2 pr: 0.8590604026845637 rec: 0.7191011235955056 f1: 0.782874617737003
#         3 pr: 0.8345323741007195 rec: 0.6705202312138728 f1: 0.7435897435897436
#         4 pr: 0.8490566037735849 rec: 0.678391959798995 f1: 0.7541899441340782
# avg: pr=0.8526857202676178 rec=0.6774632804824819 f1=0.7543716872031736

### Exploit

In [ ]:
clf_factory = lambda: CatBoostClassifier(iterations=500,
                                         learning_rate=0.2,
                                         loss_function='Logloss',
                                         verbose=False,
                                         text_features=['text', 'title'],
                                         cat_features=['type', 'site',
                                                       'zero_shot_1st_label', 'zero_shot_1st_sub_label'],
                                         feature_calcers=['BoW:top_tokens_count=2000', 'BoW:top_tokens_count=500'],
                                         )
cross_val(clf_factory, df_X, y_exp)
# Cross Validation k_folds=5, n_classes=2:
#         0 pr: 0.8433734939759037 rec: 0.813953488372093 f1: 0.8284023668639053
#         1 pr: 0.9076923076923077 rec: 0.7763157894736842 f1: 0.8368794326241134
#         2 pr: 0.8783783783783784 rec: 0.8333333333333334 f1: 0.855263157894737
#         3 pr: 0.9206349206349206 rec: 0.7733333333333333 f1: 0.8405797101449275
#         4 pr: 0.8714285714285714 rec: 0.7349397590361446 f1: 0.7973856209150327
# avg: pr=0.8843015344220163 rec=0.7863751407097178 f1=0.8317020576885431